In [1]:
import requests
import pymongo
from pymongo import MongoClient
import datetime
import time
import argparse

In [2]:
client = MongoClient('localhost', 27017)
db = client.hhemails3
hhemails3 = db.hhemails3

In [7]:
professions = 'менеджер по продажам or Sales manager or грузчик or Продавец-консультант or Продавец-консультант or повар or пекарь or Специалист колл центр or официант or водитель or продавец'


def get_vac_by_day(date):
    count = 0
    url_vac = 'https://api.hh.ru/vacancies?text=' + professions + '&per_page=100&'
    for i in range(23):
        start = i
        if len(str(start)) == 1:
            start = '0' + str(start)
        end = i + 1
        if len(str(end)) == 1:
            end = '0' + str(end)
        date_start = date + 'T' + str(start) + ':00:00'
        date_end = date + 'T' + str(end) + ':00:00'
        req = requests.get(url_vac + 'date_from=' + date_start + '&date_to=' + date_end)
        for j in range(req.json()['pages'] + 1):
            page_url = 'https://api.hh.ru/vacancies?text=' + professions + '&per_page=100&' + 'page=' + str(j) + '&'
            req = requests.get(page_url + 'date_from=' + date_start + '&date_to=' + date_end)            
            try:
                count += len(req.json()['items'])
                for k in req.json()['items']:
                    vac_id = k['id']
                    try:
                        req = requests.get('https://api.hh.ru/vacancies/' + str(vac_id)).json()
                        email = req['contacts']['email']
                        if email is not None:
                            vac_href = req['alternate_url']
                            try:
                                phones = req['contacts']['phones'][0] 
                                phone = phones['country'] + phones['city'] + phones['number']
                            except:
                                phone = ''
                            try:
                                name = req['contacts']['name']
                            except:
                                name = ''
                            try:
                                city = req['address']['city']
                            except:
                                city = ''
                            try:
                                vac_name =req['name']
                            except:
                                vac_name = ''
                            empl = {
                                "qid": str(start),
                                "email": email,
                                "vac": vac_name,
                                "phone": phone,
                                "name": name,
                                "city": city,
                                "vac_href": vac_href
                            }
                            hhemails3.insert_one(empl)
                    except:
                        pass
            except:
                print(req.json())

In [4]:
req = requests.get('https://api.hh.ru/vacancies/' + str(26038141)).json()

In [23]:
phones = req['contacts']['phones'][0] 
phone = phones['country'] + phones['city'] + phones['number']
print(phone)

74952343661


In [21]:
req['contacts']['name']

'Круглова Анна Эдуардовна'

In [16]:
req['address']['city']

'Москва'

In [18]:
req['alternate_url']

'https://career.ru/vacancy/26038141'

In [24]:
req['employer']['alternate_url']

'https://hh.ru/employer/10096'

In [6]:
req['name']

'Менеджер по продажам (B2B)'

In [2]:
client = MongoClient('localhost', 27017)
db = client.hhemails3
hhemails3 = db.hhemails3

In [3]:
hhemails3.count()

312

In [ ]:
get_vac_by_day('2018-06-01')

In [4]:
hhemails3[0]

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'hhemails3'), 'hhemails3.0')

In [ ]:
count = 0
for i in hhemails3.find():
    if i['email'] is not None:
        write_to_file(i['email'])
        

In [4]:
hhemails3.find()[1]

{'_id': ObjectId('5b1e5cd265fc9255a827372e'),
 'city': '',
 'email': 'jobsib@lenta.com',
 'name': 'Единый центр НСК',
 'phone': '78007700030',
 'qid': '00',
 'vac': 'Менеджер по продажам (улица Власихинская)',
 'vac_href': 'https://hh.ru/vacancy/25910623'}

In [8]:
hhemails3.drop()